In [ ]:
import warnings

warnings.simplefilter("ignore")

import os
import sys

folder = os.getcwd().split("\\examples")[0]
sys.path.append(folder)

from sloyka.src.data_getter import VKParser

In [2]:
owner_id='-129354225' # группа "центральный район за комфортную среду обитания"
token=''

In [3]:
posts_df = VKParser().run_posts(owner_id, token, 100, "2024-03-01")

2024-02-01 10:30:00
posts downloaded


In [4]:
posts_df.head(3)

,date,id,text,views.count,likes.count,reposts.count,type,link
0,2024-03-21 10:30:00,333530,"Сделайте сад Сан-Галли безопасным для граждан,...",2749,97,20,post,NaN
1,2024-03-20 19:30:00,333376,«Электросамокатная саранча» вновь повылезала н...,8722,168,23,post,NaN
2,2024-03-20 10:30:00,333265,Вот и отгуляла широкая Масленица! Наша команда...,4457,53,8,post,"https://vk.com/wall-129354225_332885|ярко,"


In [5]:
post_ids = posts_df["id"].to_list()
comments_df = VKParser().run_comments(owner_id=owner_id, post_ids=post_ids, access_token=token)

comments downloaded


In [6]:
comments_df.head(3)

,id,date,text,post_id,parents_stack,likes.count,type
0,333531,2024-03-21 07:35:33,Отличная идея. Территории точно хватит для реа...,333530,[],14,comment
1,333532,2024-03-21 07:39:46,Ещё для ограничения движения курьеров на элект...,333530,[],5,comment
2,333576,2024-03-21 08:37:47,"[id533640320|Андрей], через узкую калитку курь...",333530,[333532],1,comment


In [9]:
result_df = VKParser().run_parser(owner_id, token, step=100, cutoff_date='2024-03-20') #объединяем посты и комментарии в один датафрейм

2024-02-01 10:30:00
posts downloaded
comments downloaded


In [15]:
result_df.sample(5)

,date,id,text,views.count,likes.count,reposts.count,type,link,post_id,parents_stack
503,2024-03-02 17:54:57,331220,"[id588441849|Сергей], я не живу в этом доме",NaN,2,NaN,reply,NaN,331192.0,[331201]
1058,2024-02-05 16:13:46,328132,Надо конечно же восстановить исторический обли...,NaN,4,NaN,comment,NaN,327947.0,[]
731,2024-02-17 16:33:06,329439,А зачем? Сейчас же нельзя писать плохое про чи...,NaN,4,NaN,comment,NaN,329438.0,[]
298,2024-03-17 16:53:03,332868,"[id747280507|Lora], именно с согласия собстве...",NaN,1,NaN,reply,NaN,332749.0,[332759]
465,2024-03-03 09:34:13,331262,Где г/номер этого транспортного средства?,NaN,7,NaN,comment,NaN,331239.0,[]
